In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from typing import Callable

np.random.seed(42)


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "C:\Users\panuh\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\panuh\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "c:\Users\panuh\OneDrive - Nexus365\Year 4\reinforcement-learning\.venv\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\panuh\OneDrive - Nexus365\Year 4\reinforcement-learni

In [2]:
def sample_next_state(index: int, P: np.ndarray) -> int:
    probs = P[index]

    next = np.random.choice(range(P.shape[1]), p=probs)

    return int(next)

In [22]:
def td_loop(
        n_iter: int, 
        X: np.ndarray, 
        y: np.ndarray, 
        P: np.ndarray,
        link: Callable[[np.ndarray], np.ndarray], 
        inv_link: Callable[[np.ndarray], np.ndarray], 
        gamma: float, 
        alpha: float,
        epsilon: float,
    ) -> np.ndarray:
    n_samples = X.shape[0]
    n_features = X.shape[1]

    w = np.zeros(n_features)

    curr_index = int(np.random.randint(n_samples))
    curr_x = X[curr_index]
    curr_y = y[curr_index]

    i = 0
    grad = np.ones_like(w)

    while i < n_iter and np.linalg.norm(alpha * grad, 2) > epsilon:
        # Next state samples
        next_index = sample_next_state(index=curr_index, P=P)
        next_x = X[next_index]
        next_y = y[next_index]

        # Find rewards
        r = inv_link(curr_y) - gamma * inv_link(next_y)

        # TD target
        z = r + gamma * np.dot(next_x, w)

        # Find gradient
        grad = (link(np.dot(curr_x, w)) - link(z)) * curr_x

        # Update weights
        w -= alpha * grad
        
        # Update state and index
        curr_index, curr_x, curr_y = next_index, next_x, next_y
        i += 1

    return w

In [20]:
# Generate synthetic data
num_samples = 64
num_features = 3

X = np.random.randn(num_samples, num_features)
true_w = np.array([2.0, -3.5, 1.0])
y = X @ true_w + np.random.randn(num_samples) * 0.1  # Adding noise

alpha = 0.01  # Learning rate
gamma = 0.9   # Discount factor
num_iterations = 1e6  # Number of iterations
epsilon = 1e-7

P = np.ones((num_samples, num_samples)) / num_samples # Equal probability to move to any state

In [23]:
w_hat = td_loop(
    n_iter=num_iterations,
    X=X,
    y=y,
    P=P,
    link=lambda x : x,
    inv_link=lambda x : x,
    gamma=gamma,
    alpha=alpha,
    epsilon=epsilon,
)

error = np.linalg.norm(w_hat - true_w, 2)

print(w_hat)
print(f'Error: {error:.5f}')

[ 1.9918766  -3.51484946  0.98942379]
Error: 0.01996
